In [35]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [3]:
def visualize_images(input_images, num_cols, figure_name, cmap = None):
    "Shows input images by stacking them in num_cols columns"
    fig, axes = plt.subplots((int)((len(input_images) + 1) /num_cols), num_cols, figsize=(24, 20))

    fig = plt.gcf()
    fig.canvas.set_window_title(figure_name)
    
    print(figure_name)
    
    for ax, image in zip(axes.flat, input_images):
        if(cmap == "gray" or cmap == 'hot'):
            ax.imshow(image, cmap=cmap)
        elif(image.shape[2]==1):
            ax.imshow(image[:,:,0], cmap = cmap)
        else:
            ax.imshow(image, cmap=cmap)

    plt.show()
    

In [34]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    img_copy = np.copy(img)
    result_rectangles = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        area = (bbox[1][1] - bbox[0][1]) * (bbox[1][0] - bbox[0][0])
        if area > 40 * 40:
            result_rectangles.append(bbox)
            # Draw the box on the image
            # cv2.rectangle(img_copy, bbox[0], bbox[1], (0,255,0), 6)    (bbox[0][0]-100,bbox[0][1]-100), (bbox[1][0]+100,bbox[1][1]+100)
            cv2.rectangle(img_copy,(bbox[0][0]-int(0.02*bbox[0][0]),bbox[0][1]-int(0.02*bbox[0][1])), (bbox[1][0]+int(0.02*bbox[1][0]),bbox[1][1]+int(0.02*bbox[1][1])), (0,255,0), 6)
    # Return the image
    return result_rectangles, img_copy

In [26]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    random_color = False
    # Iterate through the bounding boxes
    for bbox in bboxes:
        if color == 'random' or random_color:
            color = (np.random.randint(0,255), np.random.randint(0,255), np.random.randint(0,255))
            random_color = True
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        # cv2.rectangle(imcopy, (bbox[0][0]-100,bbox[0][1]-100), (bbox[1][0]+100,bbox[1][1]+100), color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [27]:
def visualize_bboxes(image, scales = [1, 1.5, 2, 2.5, 3], 
                   ystarts = [400, 400, 450, 450, 460], 
                   ystops = [528, 550, 620, 650, 700]):
    out_rectangles = []
    svc = pickle.load(open('svc_pickle.pkl', 'rb'))
    for scale, ystart, ystop in zip(scales, ystarts, ystops):
        rectangles = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, vis_bboxes = True)
        if len(rectangles) > 0:
            out_rectangles.append(rectangles)
    out_rectangles = [item for sublist in out_rectangles for item in sublist] 

    plt.figure(figsize=(20,10))
    plt.imshow(draw_boxes(image, out_rectangles, color='random', thick=3))

In [ ]:
def combine_images( big_image, small1, small2):        
        
        background = np.zeros_like(big_image)
        large_img_size = (background.shape[1],int(0.75*background.shape[0])) 
        small_img_size=(int(background.shape[1]/2),int(0.25*background.shape[0]))
        small_img_x_offset=0 
        small_img_y_offset=0

        img2 = cv2.resize(np.dstack((small2, small2, small2))*255, small_img_size)
        img1 = cv2.resize(small1, small_img_size)
        img3 = cv2.resize(big_image, large_img_size)

        background[0: small_img_size[1], 0: small_img_size[0]] = img2
        
        start_offset_y = small_img_y_offset 
        endy = start_offset_y + small_img_size[1]
        if endy > background.shape[0]:
          endy = background.shape[0]
        start_offset_x = 2 * small_img_x_offset + small_img_size[0]
        endx = start_offset_x + small_img_size[0]
        if endx > background.shape[1]:
          endx = background.shape[1]
        background[start_offset_y:endy , start_offset_x: endx] = img1

        background[int(0.25*background.shape[0]): , : ] = img3
         
        return background